In [ ]:
%matplotlib inline
import os, shutil
import sys
sys.path.append("..")
import numpy as np
from IPython.display import Image
import pandas as pd
import matplotlib.pyplot as plt

import flopy as flopy
import pyemu

In [ ]:
import freyberg_setup as fs
fs.setup_pest_kr()
working_dir = fs.WORKING_DIR_KR
pst_name = fs.PST_NAME_KR

In [ ]:
fs.plot_model(working_dir, pst_name)

In [ ]:
# m = flopy.modflow.Modflow.load(fs.MODEL_NAM,model_ws=working_dir,load_only=["upw"],check=False)
pst = pyemu.Pst(os.path.join(working_dir,pst_name))
pst.parameter_data.loc["rch_0","partrans"] = 'log'
pst.parameter_data

### run pestpp-ies

In [ ]:
pst.pestpp_options = {"forecasts":pst.pestpp_options["forecasts"]}
pst.pestpp_options["ies_subset_size"] = 4
pst.pestpp_options["ies_lambda_mults"] = [1.0]
#pst.pestpp_options["lambda_scale_fac"] = [0.95,1.1]
pst.pestpp_options["ies_num_reals"] = 30
pst.pestpp_options["par_sigma_range"] = 1
#pst.pestpp_options["ies_reg_factor"] = 1.0
#pst.pestpp_options["parcov_filename"] = "prior.cov"
pst.control_data.noptmax = 4
pst.write(os.path.join(working_dir,pst_name))

In [ ]:
os.chdir(working_dir)
pyemu.helpers.start_slaves(".","pestpp-ies",pst_name,num_slaves=15,master_dir=".")
os.chdir("..")

In [ ]:
par = pd.read_csv(pst.filename.replace(".pst",".0.par.csv"),index_col=0)
par.columns = par.columns.map(str.lower)

In [ ]:
df0 = pd.read_csv(os.path.join(working_dir,pst_name.replace(".pst",".0.par.csv")))
df1 = pd.read_csv(os.path.join(working_dir,pst_name.replace(".pst",".1.par.csv")))

In [ ]:
colors = ["0.5","b","g",'r']
ax = plt.subplot(111)
for i,c in enumerate(colors):
    f = os.path.join(working_dir,pst_name.replace(".pst",".{0}.par.csv".format(i)))
    if not os.path.exists(f):
        break
    #print(c,f)
    df = pd.read_csv(f)    
    ax.scatter(df.HK,df.RCH_0,color=c,alpha=0.5)
    if i > 0:
        for ii in df.index:
            x = [df.loc[ii,"HK"],dflast.loc[ii,"HK"]]
            y = [df.loc[ii,"RCH_0"],dflast.loc[ii,"RCH_0"]]
            ax.plot(x,y,color=colors[i-1],alpha=0.5)
    dflast = df

In [ ]:
pyemu.plot_utils.ensemble_helper(par)

In [ ]:
init_obs = pst.filename.replace(".pst",".0.obs.csv")
final_obs = init_obs.replace("0","{0}".format(pst.control_data.noptmax))
pyemu.plot_utils.ensemble_res_1to1({'0.5':init_obs,"b":final_obs},pst=pst)

In [ ]:
init_df = pd.read_csv(init_obs,index_col=0)
init_df.columns = init_df.columns.map(str.lower)
final_df = pd.read_csv(final_obs,index_col=0)
final_df.columns = final_df.columns.map(str.lower)


In [ ]:
pst.pestpp_options

In [ ]:
pyemu.plot_utils.ensemble_helper({"0.5":init_df.loc[:,pst.forecast_names],"b":final_df.loc[:,pst.forecast_names]},
                                 deter_vals=pst.observation_data.loc[pst.forecast_names,"obsval"].to_dict(),bins=20)